In [1]:
!pip install zipfile

ERROR: Could not find a version that satisfies the requirement zipfile (from versions: none)
ERROR: No matching distribution found for zipfile


In [2]:
import torch
import torch.optim as optim
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import numpy as np
import random
from PIL import Image
import shutil
import os
import zipfile
import matplotlib.pyplot as plt
from tqdm import trange
import cv2
import glob
from sklearn.model_selection import train_test_split
from torchvision import datasets, models, transforms
from IPython.display import clear_output
import pandas as pd
import numpy as np
import torchvision

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
z = zipfile.ZipFile('/content/drive/MyDrive/archive.zip')
z.extractall()

# FaceDataset

In [5]:
main_dir = '/content/real_vs_fake/real-vs-fake/'

In [6]:
train_df = pd.read_csv('/content/train.csv')
valid_df = pd.read_csv('/content/valid.csv')
test_df = pd.read_csv('/content/test.csv')

In [7]:
train_df = train_df.sample(frac = 1)
valid_df = valid_df.sample(frac = 1)
test_df = test_df.sample(frac = 1)
train_files = np.array([main_dir + path for path in train_df['path']])
val_files = np.array([main_dir + path for path in valid_df['path']])
test_files = np.array([main_dir + path for path in test_df['path']])
train_labels = np.array([l for l in train_df['label']])
val_labels = np.array([l for l in valid_df['label']])
test_labels = np.array([l for l in test_df['label']])

In [8]:
class FaceDataset(Dataset):
      def __init__(self, files, labels, transform):
          super().__init__()
          self.data = files
          self.labels = labels
          self.transform = transform

      def __len__(self):
          return len(self.data)

      def __getitem__(self, index):
          x = cv2.imread(self.data[index])
          x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
          x = Image.fromarray(x)
          if self.transform is not None:
             x = self.transform(x)
          y = torch.tensor(self.labels[index]).long()
          return x, y

      def visualize(self, index):
          x = cv2.imread(self.data[index])
          x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
          plt.figure(figsize=(10, 10))
          plt.axis('off')
          plt.imshow(x)

In [9]:
train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

valid_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225]),
])

# Dataset and DataLoader

In [10]:
train_dataset = FaceDataset(train_files, train_labels, train_transform)
valid_dataset = FaceDataset(val_files, val_labels, valid_transform)

In [11]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64,
                                           shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                           batch_size=64,
                                           # shuffle=True,
                                           pin_memory=True,
                                           num_workers=2)

# Neptun.ai

In [12]:
%pip install neptune-client

In [13]:
from torch.optim import lr_scheduler
import neptune.new as neptune
os.environ['NEPTUNE_API_TOKEN'] = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiIwYTk5MTQwZC1mYmVmLTQ1MTItYjA0ZC05NTdjYzNjNjRhMmEifQ=='
run = neptune.init(
    api_token= os.getenv('NEPTUNE_API_TOKEN'),
    project = 'misha/fake-faces'
)

https://app.neptune.ai/misha/fake-faces/e/FAK-10
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


# Train

In [14]:
def train(train_dataloader, val_dataloader, model, epochs, batch_size):
    opt = torch.optim.Adam(model.parameters(), lr=5e-4)
    criterion = torch.nn.CrossEntropyLoss()
        
    run['model/parameters/n_epochs'] = epochs
        
    for epoch in trange(epochs, desc="Epoch:"):
        train_loss, train_acc = fit_epoch(model, train_dataloader, criterion, opt)
        val_loss, val_acc = eval_epoch(model, val_dataloader, criterion)
        run['train/epoch/loss'].log(train_loss)
        run['train/epoch/accuracy'].log(train_acc)
                      

In [15]:
def fit_epoch(model, train_loader, criterion, opt):
    model.train()
    total_acc=0
    total_loss=0
    num_pictures=0
    num_batches=0
    for x_train, y_train in train_loader:
        x_train = x_train.to(torch.device("cuda"))
        y_train = y_train.to(torch.device("cuda"))
        opt.zero_grad()
        predictions = model(x_train)
        loss = criterion(predictions, y_train)
        loss.backward()
        opt.step()
        
        pred_labels = torch.argmax(predictions, 1)
        total_acc += torch.sum(pred_labels == y_train)
        total_loss += loss.item() #mean loss for batch
        num_pictures += len(y_train) #number of pictures
        num_batches += 1 #number of batches
        run["train/accuracy"].log(torch.sum(pred_labels == y_train)/len(y_train))
        run["train/loss"].log(loss.item())
    return total_loss/num_batches, total_acc/num_pictures

In [16]:
def eval_epoch(model, val_loader, criterion):
    model.eval()
    total_acc=0
    total_loss=0
    num_pictures=0
    num_batches=0
    for x_train, y_train in val_loader:
        x_train = x_train.to(torch.device("cuda"))
        y_train = y_train.to(torch.device("cuda"))
        with torch.no_grad():
             predictions = model(x_train)
             loss = criterion(predictions, y_train)
             pred_labels = torch.argmax(predictions, 1)

             total_acc += torch.sum(pred_labels == y_train)
             total_loss += loss.item() #mean loss for batch
             num_pictures += len(y_train) #number of pictures
             num_batches += 1 #number of batches
             run["evaluation/accuracy"].log(torch.sum(pred_labels == y_train)/len(y_train))
             run["evaluation/loss"].log(loss.item())
    return total_loss/num_batches, total_acc/num_pictures

# Model

In [17]:
!pip install efficientnet_pytorch

In [18]:
from efficientnet_pytorch import EfficientNet

In [19]:
model = EfficientNet.from_pretrained('efficientnet-b3')

Loaded pretrained weights for efficientnet-b3


In [20]:
for param in  model.parameters():
    param.requires_grad=False
for param in model._blocks[22:].parameters():
    param.requires_grad=True
model._conv_head.requires_grad=True
model._bn1.requires_grad=True
model._avg_pooling.requires_grad=True
model._dropout.requires_grad=True
model._fc.requires_grad=True

In [21]:
n_classes = 2
num_features = 1536
model._fc = torch.nn.Linear(in_features=num_features, out_features=n_classes)

In [22]:
if torch.cuda.is_available():
    model = model.cuda()

# Run train

In [23]:
train(train_loader, valid_loader, model, 3, 128)

Epoch:: 100%|██████████| 3/3 [34:18<00:00, 686.32s/it]


In [30]:
torch.save(model,'model_5')

# Test predictions

In [31]:
class TestFaceDataset(Dataset):
    def __init__(self, files, transform=None):
        self.data = files
        self.transform = transform

    def __getitem__(self, idx):
        # читаем картинку
        image = cv2.imread(self.data[idx])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        
        # преобразуем, если нужно
        if self.transform:
            image = self.transform(image)
        
        return image
    
    def __len__(self):
        return len(self.data)

In [32]:
test_dataset = TestFaceDataset(test_files, valid_transform)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=64,
                                           shuffle = False,
                                           pin_memory=True,
                                           num_workers=2)

In [60]:
def make_prediction(model, test_loader):
     model.eval()
     preds = []
     for batch in test_loader:
        batch = batch.to(torch.device("cuda"))
        with torch.no_grad():
             predictions = model(batch)
             y_preds = torch.argmax(predictions, 1)
             preds.extend(y_preds.tolist())
     return preds

In [61]:
preds_list = np.array(make_prediction(model, test_loader))

In [62]:
preds_list

array([1, 1, 0, ..., 0, 0, 0])

In [63]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, preds_list)

0.988